<a href="https://colab.research.google.com/github/harryypham/MyMLPractice/blob/main/practice/eng_vi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's depende

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz

--2024-07-02 23:30:19--  https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz [following]
--2024-07-02 23:30:20--  https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9903559 (9.4M) [application/octet-stream]
Saving to: ‘train-en-vi.tgz’

train-en-vi.tgz     100%[===================>]   9.44M  --.-KB/s    in 0.02s   

2024-07-02 23:30:20 (387 MB/s) - ‘train-en-vi.tgz’ saved [9903559/9903559]



In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="vi_VN")
src_text = "How are you coping in your life in exile ?"
tgt_text = "Làm cách nào bạn đương đầu với cuộc sống lưu vong ?"
ex = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")
print(ex)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

{'input_ids': tensor([[250004,  11249,    621,    398,  11757,    214,     23,    935,   6897,
             23,   1119,   1340,    705,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[250024,  52906,   3959,   3941,   1776,  92821,   2494,   1116,   9568,
           7269,  22891,    542,    177,    705,      2]])}


In [3]:
import tarfile

file_path = 'train-en-vi.tgz'
with tarfile.open(file_path, 'r:gz') as tar:
  tar.extractall('data')

In [4]:
import os
import html
from torch.utils.data import Dataset, DataLoader

class EnToViDataset(Dataset):
  def __init__(self, root_dir, split):
    with open(os.path.join(root_dir, f'{split}.en'), 'r', encoding='utf-8') as f:
      src_data = html.unescape(f.read()).split('\n')
    with open(os.path.join(root_dir, f'{split}.vi'), 'r', encoding='utf-8') as f:
      tgt_data = html.unescape(f.read()).split('\n')
    assert len(src_data) == len(tgt_data), 'Source and target file have different number of lines'
    self.data = list(zip(src_data, tgt_data))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]



In [ ]:
train_data = EnToViDataset('data', 'train')
print(train_data.__getitem__(2))
train_loader = DataLoader(train_data, batch_size=2, shuffle=True, pin_memory=True)
batch = next(iter(train_loader))
print(batch)

("I 'd like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .", 'Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy trên báo .')
[('How are you coping in your life in exile ?', 'This repeats itself through the night -- rehearsal , tying to the mast , conning his way out of it , beating the poor first mate up mercilessly .'), ('Làm cách nào bạn đương đầu với cuộc sống lưu vong ?', 'Nó cứ tiếp tục lặp đi lặp lại như thế suốt đêm diễn tập , trói vào cột buồm tìm cách thoát ra , đánh đập viên thuyền phó tàn bạo .')]


In [5]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

with open(os.path.join('data', f'train.en'), 'r', encoding='utf-8') as f:
      src_data = html.unescape(f.read()).split('\n')
with open(os.path.join('data', f'train.vi'), 'r', encoding='utf-8') as f:
      tgt_data = html.unescape(f.read()).split('\n')

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50", use_fast=True, src_lang="en_XX", tgt_lang="vi_VN")

def data_gen():
  for idx, (src, tgt) in enumerate(zip(src_data, tgt_data)):
    yield tokenizer(src, text_target=tgt, max_length=128, truncation=True)

train_data = Dataset.from_generator(data_gen)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(train_data[0])
tokenizer.decode(train_data[0]['input_ids'], skip_special_tokens=True)

{'input_ids': [250004, 127055, 66937, 13, 152, 581, 41664, 50155, 10, 153552, 10336, 2256, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [250024, 67766, 2546, 218877, 858, 889, 10037, 6248, 1893, 17964, 42254, 2]}


'Rachel Pike : The science behind a climate headline'

In [6]:
from transformers import DataCollatorForSeq2Seq

checkpoint = "facebook/mbart-large-50"
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

train_data.set_format('torch')
train_loader = DataLoader(train_data, shuffle=True, batch_size=8, collate_fn=data_collator, pin_memory=True)

In [ ]:
import numpy as np
batch = next(iter(train_loader))
print(batch['labels'].shape)
print(tokenizer.lang_code_to_id["vi_VN"])
# vi_tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50", use_fast=True, src_lang="vi_VN", tgt_lang="en_XX")
labels = np.where(batch['labels'] != -100, batch['labels'], tokenizer.pad_token_id)
print(tokenizer.batch_decode(labels, skip_special_tokens=True))

torch.Size([4, 33])
250024
['chúng ta nên thực tế một chút', 'Vâng bạn, hôm nay, phổ cập các nguyên tắc của bạn.', 'Họ đối mặt với nhà cầm quyền.', 'Nếu ai đó nói " Ồ, tôi thấy bức tượng chân dung, " nó có thể không còn là bức tượng sau vài tháng.']


In [7]:
import evaluate

metric = evaluate.load("sacrebleu")

In [8]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [36]:
from transformers import MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained(checkpoint)
print(model.get_memory_footprint())

2444518168


In [12]:
print(model.num_parameters())

610879488


In [30]:
from functools import partial
import torch
import math
from torch.optim.lr_scheduler import LambdaLR

def _get_cosine_schedule_with_warmup_lr_lambda(
    current_step: int, *, num_warmup_steps: int, num_training_steps: int, num_cycles: float
):
    print('update lr...')
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1, num_warmup_steps))
    progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
    return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

def get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps: int, num_training_steps: int, num_cycles: float = 0.5, last_epoch: int = -1
):
    # lr_lambda = partial(
    #     _get_cosine_schedule_with_warmup_lr_lambda,
    #     num_warmup_steps=num_warmup_steps,
    #     num_training_steps=num_training_steps,
    #     num_cycles=num_cycles,
    # )
    lr_func = lambda iter: torch.tensor(_get_cosine_schedule_with_warmup_lr_lambda(iter,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
        num_cycles=num_cycles,), requires_grad=False)
    return LambdaLR(optimizer, lr_func, last_epoch)

In [37]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
optimizer = AdamW(model.parameters(), lr=2e-5)
dtype = dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
scaler = torch.cuda.amp.GradScaler(init_scale=2.0**14, enabled=(dtype == 'float16'))
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer, num_warmup_steps=50, num_training_steps=num_training_steps
)


update lr...


In [38]:
torch.cuda.empty_cache()
# torch.autograd.set_detect_anomaly(True)
print('compiling model...')
model = torch.compile(model)
model.to(device)

print(f'training with {dtype}')
pbar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
  for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with ctx:
          outputs = model(**batch)
          loss = outputs.loss

        #loss.backward()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        for name, param in model.named_parameters():
          if torch.isnan(param.grad).any():
            print("nan gradient found")

        scaler.step(optimizer)
        # optimizer.step()
        scale = scaler.get_scale()
        print(scale)
        scaler.update()
        if scaler.get_scale() >= scale:
          lr_scheduler.step()

        optimizer.zero_grad()
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})

compiling model...
training with bfloat16


  0%|          | 0/33330 [00:00<?, ?it/s]

update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...
update lr...

KeyboardInterrupt: 

In [39]:
text = 'Swift spent her early years on a Christmas tree farm in New York that her father had purchased from one of his clients, and she spent her summers at her family\'s vacation home in, New Jersey, where she occasionally performed acoustic songs at a local coffee shop.'
input_tokens = tokenizer(text, max_length=128, truncation=True, return_tensors="pt").to(device)
print(input_tokens)
generated_tokens = model.generate(**input_tokens, forced_bos_token_id=tokenizer.lang_code_to_id["vi_VN"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

{'input_ids': tensor([[250004, 130643,  66933,    604,  39395,   5369,     98,     10,  40111,
          53201,  40617,     23,   2356,   5753,    450,    604,  67373,   1902,
          59038,     71,   1295,   1632,    111,   1919,  19802,      4,    136,
           2412,  66933,    604,  51065,      7,     99,    604,  14449,     25,
              7, 166021,   5368,     23,      4,   2356, 101033,      4,   7440,
           2412,  46505,  25958,  51339,    297,     10,  12600,  28692,  52336,
             99,     10,   4000,  79497,  19927,      5,      2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
['Trong những năm đầu tiên của mình ở New York, cô ấy đã mua 1 cây Christmas tree từ một khách hàng của ông ta và cô ấy đã nghỉ hè tại nhà của mình ở New Jersey

In [ ]:

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"
article = "UN Chief Says There Is No Military Solution in Syria"
inputs = tokenizer(article, return_tensors="pt")


tensor([[     2, 250024,  19167,  29004,   2494,  25332, 140857,   8735,   2872,
            687,    524,   8652,   6800,  29225,   2550,   2059,  51712,      2]])


['Người đứng đầu Liên Hiệp Quốc nói không có giải pháp quân sự ở Syria']

In [ ]:
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["vi_VN"], output_scores=True)
print(translated_tokens)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

tensor([[     2, 250024,  19167,  29004,   2494,  25332, 140857,   8735,   2872,
            687,    524,   8652,   6800,  29225,   2550,   2059,  51712,      2]])


['Người đứng đầu Liên Hiệp Quốc nói không có giải pháp quân sự ở Syria']